In [42]:
import rasterio
from rasterstats import zonal_stats
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import warnings
from shapely.errors import ShapelyDeprecationWarning
# Suppress specific ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [43]:
# Set dataset paths
vector_path = r"D:\projects\sovereign-risk\Thailand\data\flood\basins\analysis_basins.gpkg"
risk_map_dir = r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps"

In [44]:
rasters_info = [
    {"epoch": "Today", "adaptation_scenario": "Baseline", "RP": 2, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\present_2_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Baseline", "RP": 5, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\present_5_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Baseline", "RP": 10, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\present_10_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Baseline", "RP": 25, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\present_25_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Baseline", "RP": 50, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\present_50_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Baseline", "RP": 100, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\present_100_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Baseline", "RP": 200, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\present_200_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Baseline", "RP": 500, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\present_500_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Baseline", "RP": 1000, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\present_1000_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Baseline", "RP": 2, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\present_2_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Baseline", "RP": 5, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\present_5_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Baseline", "RP": 10, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\present_10_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Baseline", "RP": 25, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\present_25_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Baseline", "RP": 50, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\present_50_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Baseline", "RP": 100, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\present_100_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Baseline", "RP": 200, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\present_200_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Baseline", "RP": 500, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\present_500_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Baseline", "RP": 1000, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\present_1000_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Baseline", "RP": 2, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_h_2_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Baseline", "RP": 5, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_h_5_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Baseline", "RP": 10, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_h_10_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Baseline", "RP": 25, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_h_25_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Baseline", "RP": 50, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_h_50_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Baseline", "RP": 100, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_h_100_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Baseline", "RP": 200, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_h_200_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Baseline", "RP": 500, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_h_500_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Baseline", "RP": 1000, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_h_1000_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Baseline", "RP": 2, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_h_2_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Baseline", "RP": 5, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_h_5_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Baseline", "RP": 10, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_h_10_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Baseline", "RP": 25, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_h_25_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Baseline", "RP": 50, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_h_50_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Baseline", "RP": 100, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_h_100_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Baseline", "RP": 200, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_h_200_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Baseline", "RP": 500, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_h_500_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Baseline", "RP": 1000, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_h_1000_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Baseline", "RP": 2, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_l_2_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Baseline", "RP": 5, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_l_5_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Baseline", "RP": 10, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_l_10_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Baseline", "RP": 25, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_l_25_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Baseline", "RP": 50, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_l_50_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Baseline", "RP": 100, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_l_100_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Baseline", "RP": 200, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_l_200_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Baseline", "RP": 500, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_l_500_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Baseline", "RP": 1000, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_l_1000_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Baseline", "RP": 2, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_l_2_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Baseline", "RP": 5, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_l_5_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Baseline", "RP": 10, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_l_10_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Baseline", "RP": 25, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_l_25_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Baseline", "RP": 50, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_l_50_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Baseline", "RP": 100, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_l_100_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Baseline", "RP": 200, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_l_200_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Baseline", "RP": 500, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_l_500_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Baseline", "RP": 1000, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\baseline\future_l_1000_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Dry_Proofing", "RP": 2, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\present_2_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Dry_Proofing", "RP": 5, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\present_5_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Dry_Proofing", "RP": 10, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\present_10_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Dry_Proofing", "RP": 25, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\present_25_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Dry_Proofing", "RP": 50, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\present_50_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Dry_Proofing", "RP": 100, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\present_100_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Dry_Proofing", "RP": 200, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\present_200_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Dry_Proofing", "RP": 500, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\present_500_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Dry_Proofing", "RP": 1000, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\present_1000_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Dry_Proofing", "RP": 2, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\present_2_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Dry_Proofing", "RP": 5, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\Dry_Proofing\present_5_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Dry_Proofing", "RP": 10, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\present_10_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Dry_Proofing", "RP": 25, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\present_25_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Dry_Proofing", "RP": 50, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\present_50_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Dry_Proofing", "RP": 100, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\present_100_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Dry_Proofing", "RP": 200, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\present_200_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Dry_Proofing", "RP": 500, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\present_500_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Dry_Proofing", "RP": 1000, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\present_1000_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 2, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_h_2_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 5, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_h_5_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 10, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_h_10_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 25, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_h_25_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 50, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_h_50_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 100, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_h_100_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 200, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_h_200_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 500, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_h_500_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 1000, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_h_1000_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 2, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_h_2_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 5, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_h_5_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 10, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_h_10_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 25, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_h_25_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 50, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_h_50_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 100, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_h_100_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 200, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_h_200_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 500, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_h_500_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 1000, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_h_1000_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 2, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_l_2_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 5, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_l_5_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 10, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_l_10_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 25, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_l_25_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 50, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_l_50_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 100, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_l_100_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 200, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_l_200_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 500, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_l_500_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 1000, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_l_1000_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 2, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_l_2_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 5, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_l_5_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 10, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_l_10_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 25, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_l_25_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 50, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_l_50_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 100, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_l_100_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 200, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_l_200_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 500, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_l_500_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Dry_Proofing", "RP": 1000, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\dry_Proofing\future_l_1000_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Urban_Protection_RP100", "RP": 2, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\present_2_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Urban_Protection_RP100", "RP": 5, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\present_5_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Urban_Protection_RP100", "RP": 10, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\present_10_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Urban_Protection_RP100", "RP": 25, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\present_25_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Urban_Protection_RP100", "RP": 50, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\present_50_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Urban_Protection_RP100", "RP": 100, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\present_100_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Urban_Protection_RP100", "RP": 200, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\present_200_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Urban_Protection_RP100", "RP": 500, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\present_500_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Urban_Protection_RP100", "RP": 1000, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\present_1000_res_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Urban_Protection_RP100", "RP": 2, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\present_2_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Urban_Protection_RP100", "RP": 5, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\present_5_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Urban_Protection_RP100", "RP": 10, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\present_10_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Urban_Protection_RP100", "RP": 25, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\present_25_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Urban_Protection_RP100", "RP": 50, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\present_50_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Urban_Protection_RP100", "RP": 100, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\present_100_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Urban_Protection_RP100", "RP": 200, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\present_200_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Urban_Protection_RP100", "RP": 500, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\present_500_nres_damages.tif"},
    {"epoch": "Today", "adaptation_scenario": "Urban_Protection_RP100", "RP": 1000, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\present_1000_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 2, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_h_2_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 5, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_h_5_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 10, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_h_10_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 25, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_h_25_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 50, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_h_50_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 100, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_h_100_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 200, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_h_200_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 500, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_h_500_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 1000, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_h_1000_res_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 2, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_h_2_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 5, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_h_5_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 10, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_h_10_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 25, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_h_25_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 50, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_h_50_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 100, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_h_100_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 200, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_h_200_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 500, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_h_500_nres_damages.tif"},
    {"epoch": "Future_High_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 1000, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_h_1000_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 2, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_l_2_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 5, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_l_5_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 10, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_l_10_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 25, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_l_25_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 50, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_l_50_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 100, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_l_100_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 200, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_l_200_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 500, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_l_500_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 1000, "urban_class": "Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_l_1000_res_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 2, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_l_2_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 5, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_l_5_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 10, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_l_10_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 25, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_l_25_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 50, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_l_50_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 100, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_l_100_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 200, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_l_200_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 500, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_l_500_nres_damages.tif"},
    {"epoch": "Future_Low_Emission", "adaptation_scenario": "Urban_Protection_RP100", "RP": 1000, "urban_class": "Non-Residential", "file": r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\infrastructure\urban_rp100\future_l_1000_nres_damages.tif"}
]

In [45]:
# Laod analysis basins
analysis_basins = gpd.read_file(vector_path)
# Add an index column for good measure
analysis_basins.reset_index(drop=True, inplace=True)
analysis_basins['ID'] = analysis_basins.index + 1

In [46]:
# Initialize empty dataframe for storing the results
columns = ['zone_id', 'RP', 'epoch', 'adaptation_scenario', 'urban_class', 'damages'] + list(analysis_basins.columns)
results_df = pd.DataFrame(columns=columns)

In [47]:
results_df

,zone_id,RP,epoch,adaptation_scenario,urban_class,damages,HYBAS_ID_04,HYBAS_ID_05,HYBAS_ID_06,HYBAS_ID_07,...,NAME,MerL_Riv,r_lng_km,u_r_lng_km,Add_Pr,New_Pr_L,Add_Pr_c,Add_Pr_c_u,geometry,ID


In [52]:
# Iterate over raster files, compute zonal statistics for each and store results
# Initialize empty list to collect DataFrames
dfs_list = []

i = 0
for raster_info in rasters_info:
    with rasterio.open(raster_info['file']) as src:
        nodata = src.nodata
        if nodata is None:
            nodata = 0
        affine = src.transform
        array = src.read(1, masked=True)
        array.fill_value = nodata  # Ensure the fill value for masked elements is the no-data value
        array = array.filled()  # Fill masked (no-data) values with the specified no-data value
        stats = zonal_stats(analysis_basins, array, affine=affine, stats="sum", band=1, all_touched=False, nodata=nodata) # only count pixels whose centre is within the polygon

    for idx, zone_stat in enumerate(stats):
        # Construct a new DataFrame for each row
        df_row = pd.DataFrame({
            'zone_id': [zone_stat['ID']],
            'RP': [raster_info['RP']],
            'epoch': [raster_info['epoch']],
            'adaptation_scenario': [raster_info['adaptation_scenario']],
            'urban_class': [raster_info['urban_class']],
            'damages_USD': [zone_stat['sum']]
        })

        # Append the DataFrame to the dfs_list
        dfs_list.append(df_row)
        
            
    
    i += 1
    if i > 1:
        break
# Concatenate all DataFrames in the list to form the final results DataFrame
results_df = pd.concat(dfs_list, ignore_index=True)
            

KeyError: 'ID'

In [118]:
# Can we do zonal statistics on this???
from rasterstats import zonal_stats

In [119]:
vector_path_country = r"D:\projects\sovereign-risk\Thailand\data\GADM\gadm36_THA_shp\gadm36_THA_0.shp"
vector_path_analysis = r"D:\projects\sovereign-risk\Thailand\data\flood\basins\analysis_basins.gpkg"

In [120]:
country = gpd.read_file(vector_path_country)
basins = gpd.read_file(vector_path_analysis)

In [121]:
# Perform zonal statistics
stats_country = zonal_stats(country, output_path, stats="sum", band=1, all_touched=False) # only count pixels whos center is within the polgyon
stats_basins = zonal_stats(basins, output_path, stats="sum", band=1, all_touched=False)

In [124]:
country['Risk'] = [stat['sum'] for stat in stats_country]
basins['Risk'] = [stat['sum'] for stat in stats_basins]

In [125]:
country.to_file(r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\test_country.gpkg")
basins.to_file(r"D:\projects\sovereign-risk\Thailand\analysis\flood\risk_maps\test_basins.gpkg")